In [1]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 4.0 MB/s 
     |████████████████████████████████| 46 kB 1.7 MB/s 
     |████████████████████████████████| 86 kB 1.9 MB/s 


In [2]:
!pip install insightface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 438 kB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 13.5 MB 68.1 MB/s 
     |████████████████████████████████| 1.0 MB 48.6 MB/s 
  Created wheel for insightface: filename=insightface-0.7.1-cp38-cp38-linux_x86_64.whl size=989248 sha256=a0521234f9d0260f2c07665a0be54432b14d60d4ee486bcbdd5dfdea897a6cff
  Stored in directory: /root/.cache/pip/wheels/2c/dc/f8/8f4629b9bebd920d8f74a46bcd78c9359015e447a5c0d91e7a
Successfully built insightface
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages

In [ ]:
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

In [ ]:
ins_detector = FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [04:34<00:00, 1026.79KB/s]
/usr/local/lib/python3.8/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5


In [ ]:
ins_detector.prepare(ctx_id=0, det_size=(640, 640))

set det-size: (640, 640)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_meta_data(faces):
  data = []
  attributes = ['bbox', 'kps', 'det_score']
  for index in range(len(faces)):
    meta_data = {}
    for attr in attributes:
      meta_data[attr] = faces[index][attr]
    data.append(meta_data)
  return data

In [ ]:
def get_bbox_insightface(faces):
  res_faces = []
  for index in range(len(faces)):
    res = tuple(faces[index]['bbox'])
    res = tuple(map(round, res))
    res = (max(res[1],0), max(res[3],0), max(res[0], 0), max(res[2],0))
    res_faces.append(res)
  return res_faces

In [ ]:
def detect_face_ins(img_path):
  # phải loại bỏ phần .png, .jpg của tấm hình đầu vào
  # nguyên nhân là do yêu cầu của insightface
  img_path = "".join(img_path.split(".")[:-1])

  # Chạy insightface
  img = ins_get_image(img_path)
  faces = ins_detector.get(img)

  # Lấy những thông tin liên quan
  meta_data = get_meta_data(faces)
  # Chỉnh lại bbox
  res_faces = get_bbox_insightface(faces)
  return res_faces

In [ ]:
import cv2
import os

In [ ]:
def read_img(filepath, mode="RGB"):
    """
    Function to read image from file and convert it to RGB by default, or leaves BGR.
    Input: file path.
    Output: image as numpy array.
    """
    img = cv2.imread(filepath)
    if mode == "RGB":
        # Convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
def get_roi(image, coordinates):
    """
    Function to extract region of interest (roi) from image using coordinates.
    Input image: image as numpy array to extract from.
    Input coordinates: coordinates of roi in form of tuple (top, bottom, left, right)
    """
    # Check negative coordinates
    top = coordinates[0] if coordinates[0] - 5 < 0 else coordinates[0] - 5 
    bottom = coordinates[1] + 5
    left = coordinates[2] if coordinates[2] - 5 < 0 else coordinates[2] - 5
    right = coordinates[3] + 5
    # roi = image[coordinates[0]-5:coordinates[1]+5, coordinates[2]-5:coordinates[3]+5]
    roi = image[top:bottom,left:right]
    return roi

In [ ]:
def extract_face(input_path,output_path):
  scenename = input_path.split('/')[9]
  shotname = input_path.split('/')[10]
  framename = input_path.split('/')[-1].split('.')[0]
  moviename = scenename.split('-')[0]

  print(framename)

  directory_path = os.path.join(output_path,moviename,scenename,shotname,framename)
  if os.path.exists(directory_path):
    print("Thu muc da ton tai ", directory_path)
    return False
    
  if os.path.isdir(directory_path) == False:
    os.makedirs(directory_path)

  count = 0
  res_faces = detect_face_ins(input_path)
  for face in res_faces:
    #result = cv2.imread(get_roi(read_img(input_path),face))
    cv2.imwrite(os.path.join(directory_path,"{}-face_{}.jpg".format(framename,str(count))),cv2.cvtColor(get_roi(read_img(input_path),face),cv2.COLOR_RGB2BGR))
    count += 1

In [ ]:
def take_all_dir(video_dir):
  scene_dirs = [os.path.join(video_dir, file) for file in os.listdir(video_dir)]
  all_dirs = []
  count = 0
  for scene_dir in scene_dirs:
    shot_dirs = [os.path.join(scene_dir, file) for file in os.listdir(scene_dir)]
    all_dirs += shot_dirs
  return all_dirs

In [ ]:
def run_extract(frame_path):
  output_path = "/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/faces_5fps/"
  data = take_all_dir(frame_path)
  for frame in data:
    extract_face(frame,output_path)

In [ ]:
import argparse

def main():
  parser = argparse.ArgumentParser(description='Optional description')
  parser.add_argument('id', type=int,
                    help='ID of input frame')
  args = parser.parse_args()

  run_extract(args.id)

In [ ]:
if __name__ == '__main__':
    # main()
  #frame_dir = "/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/frames_5fps/Calloused_Hands/Calloused_Hands-22/"
  #frame_paths = [os.path.join(frame_dir, file) for file in os.listdir(frame_dir)]
  movie_dir = "/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/frames_5fps/Liberty_Kid/"
  list_scenes = sorted(os.listdir(movie_dir))
  for scene in list_scenes:
    frame_dir= os.path.join(movie_dir, scene)
    run_extract(frame_dir)

Streaming output truncated to the last 5000 lines.
Thu muc da ton tai  /content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/faces_5fps/Liberty_Kid/Liberty_Kid-40/Liberty_Kid-40-shot_5/Liberty_Kid-40-shot_5-frame_11
Liberty_Kid-40-shot_5-frame_12
Thu muc da ton tai  /content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/faces_5fps/Liberty_Kid/Liberty_Kid-40/Liberty_Kid-40-shot_5/Liberty_Kid-40-shot_5-frame_12
Liberty_Kid-40-shot_5-frame_13
Thu muc da ton tai  /content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/faces_5fps/Liberty_Kid/Liberty_Kid-40/Liberty_Kid-40-shot_5/Liberty_Kid-40-shot_5-frame_13
Liberty_Kid-40-shot_5-frame_14
Thu muc da ton tai  /content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/faces_5fps/Liberty_Kid/Liberty_Kid-40/Liberty_Kid-40-shot_5/Liberty_Kid-40-shot_5-frame_14
Liberty_Kid-40-shot_5-frame_15
Thu muc da ton tai  /content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/faces_5fps/Liberty_Kid/Liberty_

/usr/local/lib/python3.8/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Liberty_Kid-52-shot_9-frame_63
Liberty_Kid-52-shot_9-frame_64
Liberty_Kid-52-shot_9-frame_65
Liberty_Kid-52-shot_9-frame_66
Liberty_Kid-52-shot_9-frame_67
Liberty_Kid-52-shot_9-frame_68
Liberty_Kid-52-shot_9-frame_69
Liberty_Kid-52-shot_9-frame_70
Liberty_Kid-52-shot_9-frame_71
Liberty_Kid-52-shot_9-frame_72
Liberty_Kid-52-shot_9-frame_73
Liberty_Kid-52-shot_9-frame_74
Liberty_Kid-52-shot_9-frame_75
Liberty_Kid-52-shot_9-frame_76
Liberty_Kid-52-shot_9-frame_77
Liberty_Kid-52-shot_9-frame_78
Liberty_Kid-52-shot_9-frame_79
Liberty_Kid-52-shot_9-frame_80
Liberty_Kid-52-shot_9-frame_81
Liberty_Kid-52-shot_9-frame_82
Liberty_Kid-52-shot_9-frame_83
Liberty_Kid-52-shot_9-frame_84
Liberty_Kid-52-shot_9-frame_85
Liberty_Kid-52-shot_9-frame_86
Liberty_Kid-52-shot_9-frame_87
Liberty_Kid-52-shot_9-frame_88
Liberty_Kid-52-shot_9-frame_89
Liberty_Kid-52-shot_9-frame_90
Liberty_Kid-52-shot_9-frame_91
Liberty_Kid-52-shot_9-frame_92
Liberty_Kid-5